In [38]:
from pyannote.audio import Pipeline
from transformers import pipeline as transcribe_pipeline
import torchaudio
from huggingface_hub import login
from config import HF_TOKEN



In [39]:
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/mpham/.cache/huggingface/token
Login successful


In [40]:
# Initialize pyannote pipeline for speaker diarization
print("loading pyannote pipeline")
diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

# Initialize Whisper model for ASR (automatic speech recognition)
print("loading whisper model")
whisper_model = transcribe_pipeline("automatic-speech-recognition", model="openai/whisper-base")

# Path to your audio file
audio_file = "09-07-2017 audio_6 APCNV-2016-565.mp3"

# Perform diarization
print("performing diarization")
diarization = diarization_pipeline(audio_file)

# Load the audio file
print("loading audio file")
waveform, sample_rate = torchaudio.load(audio_file)

# Process each segment and transcribe
transcriptions = []
print("processing segments")
for segment in diarization.itertracks(yield_label=True):
    start_time = segment[0].start
    end_time = segment[0].end
    speaker_label = segment[2]
    
    # Extract the segment from the waveform
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)
    segment_waveform = waveform[:, start_sample:end_sample]

    # Convert the segment to text using Whisper
    transcription = whisper_model(segment_waveform.squeeze().numpy(), return_timestamps=False)
    transcriptions.append(f"Speaker {speaker_label}: {transcription['text']}")
    print(f"Speaker {speaker_label}: {transcription['text']}")

# Print the transcriptions
for transcription in transcriptions:
    print(transcription)

loading pyannote pipeline


loading whisper model


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


performing diarization
